CF 기반 추천시스템 구현

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

import warnings

warnings.filterwarnings('ignore')

In [2]:
!pip install tmp

  Preparing metadata (setup.py) ... done
  Created wheel for tmp: filename=tmp-0.0.2-py3-none-any.whl size=1983 sha256=2bc203110ce486a60bb7e41d6db3c4129ef08f7d137709a90faf0b2d9cae51ce
  Stored in directory: /Users/isoyeon/Library/Caches/pip/wheels/4d/82/69/f80b33540fb3af9f51862dff74e6ac92c764d78ec5f38d6c3b
Successfully built tmp


In [4]:
data = pd.read_csv('./맥주전처리후데이터_도수.csv', encoding='utf-8', index_col=0)

In [5]:
# n개 이상의 데이터 리뷰를 남긴 유저, 맥주를 걸러내는 함수
def preprocessing(data, n):
    min_id = data['아이디'].value_counts() >= n
    min_id = min_id[min_id].index.to_list()
    data = data[data['아이디'].isin(min_id)]

    min_beer = data['맥주'].value_counts() >= n
    min_beer = min_beer[min_beer].index.to_list()
    data = data[data['맥주'].isin(min_beer)]

    return data

tmp = data.copy()
temp=tmp.copy()

# 10번 반복합니다.
for i in range(1,10):
    temp = preprocessing(temp, 10)
    print(temp.shape)

temp.to_csv('정제된데이터.csv', encoding='utf-8')

(44779, 10)
(44752, 10)
(44752, 10)
(44752, 10)
(44752, 10)
(44752, 10)
(44752, 10)
(44752, 10)
(44752, 10)


맥주간 유사도 기반 추천

In [7]:
temp.shape

(44752, 10)

In [6]:
data = pd.read_csv('정제된데이터.csv', encoding='utf-8', index_col=0)

ratings = data.copy()

# 피벗 테이블을 이용해 유저-아이디 매트릭스 구성
ratings_matrix = ratings.pivot_table('평점', index='아이디', columns='맥주')
ratings_matrix.head(3)
# fillna함수를 이용해 Nan처리
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

맥주,ARK Be High IPA,ARK Black Swan,ARK Hug Me,Amazing Shocking Stout,Asahi Super Dry,Asahi Super Dry Black,Ballast Point Sculpin IPA,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,...,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,The Hand and Malt Slow IPA,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0
"00cobraR(1,103)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0
0o9i(439),0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1010(309),0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12HundredBaud(262),0.0,0.0,0.0,0.0,2.3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zomba(192),0.0,0.0,0.0,0.0,2.2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombywoof(537),0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"zvikar(12,403)",0.0,3.3,0.0,0.0,0.0,2.5,4.0,2.1,0.0,0.0,...,0.0,0.0,0.0,1.7,0.0,1.7,0.0,1.4,0.0,0.0


In [10]:
# 유사도 계산을 위해 트랜스포즈
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T

아이디,007lund(92),"00cobraR(1,103)",0o9i(439),1010(309),12HundredBaud(262),16erBlech(560),"17thfloor(2,444)","17wojownik(1,450)",184601(206),"19Ei8hty(1,167)",...,"zdk(1,917)","zebracakes(1,339)","zenkis(1,051)","zgaier(3,134)",ziobarba2000(119),zomba(192),zombywoof(537),"zvikar(12,403)","zvsn(7,783)","zvsn(7,795)"
맥주,,,,,,,,,,,,,,,,,,,,,
ARK Be High IPA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARK Black Swan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3,0.0,0.0
ARK Hug Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Amazing Shocking Stout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Asahi Super Dry,0.0,0.0,0.0,0.0,2.3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.2,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tsingtao,2.5,2.1,0.0,0.0,0.0,3.0,2.5,2.1,0.9,0.0,...,1.8,1.7,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0
Tsingtao Wheat Beer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Victoria Bitter,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0


In [11]:
# 아이템-유저 매트릭스로부터 코사인 유사도 구하기
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬에 맥주명을 매핑해 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(85, 85)


맥주,ARK Be High IPA,ARK Black Swan,ARK Hug Me,Amazing Shocking Stout,Asahi Super Dry,Asahi Super Dry Black,Ballast Point Sculpin IPA,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,...,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,The Hand and Malt Slow IPA,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
맥주,,,,,,,,,,,,,,,,,,,,,
ARK Be High IPA,1.000000,0.565494,0.640645,0.147848,0.066085,0.108746,0.076826,0.064768,0.059240,0.118815,...,0.013003,0.146517,0.249529,0.049537,0.240113,0.032552,0.218243,0.074534,0.0,0.031667
ARK Black Swan,0.565494,1.000000,0.331260,0.262579,0.065719,0.157048,0.079410,0.066557,0.056724,0.061086,...,0.014558,0.100280,0.277237,0.058194,0.184536,0.043192,0.191813,0.042416,0.0,0.037998
ARK Hug Me,0.640645,0.331260,1.000000,0.138757,0.076670,0.151207,0.064047,0.048824,0.037877,0.089515,...,0.011003,0.113863,0.149920,0.020373,0.257586,0.027855,0.118776,0.081444,0.0,0.056842


In [12]:
ratings_matrix.columns

Index(['ARK Be High IPA', 'ARK Black Swan', 'ARK Hug Me',
       'Amazing Shocking Stout', 'Asahi Super Dry', 'Asahi Super Dry Black',
       'Ballast Point Sculpin IPA', 'Beck's',
       'Berliner Kindl Jubiläums Pilsener Premium', 'Bintang Pilsener',
       'Blue Girl (5.0%)', 'Blue Moon Belgian White', 'Budweiser',
       'Carlsberg Pilsner', 'Cass Fresh', 'Cass Lemon', 'Cass Light',
       'Castlemaine XXXX Gold Lager', 'Desperados', 'Desperados Lime',
       'Desperados Mojito', 'Desperados Red', 'Edelweiss Weissbier Snowfresh',
       'Egger Märzenbier', 'Erdinger Weissbier', 'Erdinger Weissbier Dunkel',
       'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
       'Gambrinus Originál 10° (Desítka)', 'Goose Island 312 Urban Wheat Ale',
       'Goose Island Goose IPA', 'Grimbergen Cuvée Blanche',
       'Grolsch Premium Lager', 'Guinness Draught',
       'Guinness Original 4.2% (Ireland/UK)', 'Harbin Beer', 'Heineken',
       'Heineken Dark Lager', 'Heretic Evil Twin', 'Hite 

In [17]:
# 코젤 맥주와 유사도가 높은 맥주 5개만 추출하기
item_sim_df['Kozel Černý (Dark) 10°'].sort_values(ascending=False)[:5]

맥주
Kozel Černý (Dark) 10°              1.000000
Gambrinus Originál 10° (Desítka)    0.525178
Paulaner Hefe-Weissbier             0.498137
Krombacher Weizen                   0.482808
Peroni Nastro Azzurro               0.470567
Name: Kozel Černý (Dark) 10°, dtype: float64

In [14]:
# 호가든 맥주와 유사도가 높은 맥주 5개만 추출하기
item_sim_df['Hoegaarden'].sort_values(ascending=False)[:5]

맥주
Guinness Draught    1.000000
Hoegaarden          0.806490
Pilsner Urquell     0.798197
Stella Artois       0.763984
Heineken            0.696075
Name: Guinness Draught, dtype: float64

개인화된 맥주 추천

In [18]:
# ratings_arr.dot(item_sim_arr)는 평점 * 맥주 유사도
# ratings_arr는 사용자 u의 아이템 i와 가장 유사도가 높은 Top_N개 아이템에 대한 실제 평점 벡터
# item_sim_arr는 아이템 i와 가장 유사도가 높은 Top_N개 아이템의 유사도 벡터
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

# 개인화된 예측 평점 구하기
# 평점 value와 유사도 value만 뽑아서 대입
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns = ratings_matrix.columns)

# 개인별로 계산된 예측 평점
ratings_pred_matrix

맥주,ARK Be High IPA,ARK Black Swan,ARK Hug Me,Amazing Shocking Stout,Asahi Super Dry,Asahi Super Dry Black,Ballast Point Sculpin IPA,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,...,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,The Hand and Malt Slow IPA,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.142545,0.176231,0.157959,0.155443,0.572987,0.308749,0.627756,0.734217,0.421632,0.448787,...,0.348965,0.407153,0.120126,0.644577,0.183878,0.742025,0.185599,0.512479,0.273117,0.551818
"00cobraR(1,103)",0.209848,0.252822,0.209273,0.246493,0.689496,0.372706,0.860851,0.867126,0.523924,0.553536,...,0.390419,0.497462,0.185003,0.775651,0.233373,0.870910,0.260204,0.613191,0.321158,0.670384
0o9i(439),0.158681,0.198786,0.169981,0.251206,0.555514,0.331343,0.801749,0.681260,0.424414,0.408579,...,0.270459,0.405674,0.193467,0.618928,0.187409,0.668665,0.178496,0.473633,0.270450,0.543851
1010(309),0.169199,0.201604,0.181283,0.204839,0.559396,0.306278,0.723009,0.717827,0.420399,0.445245,...,0.259822,0.394400,0.163406,0.625444,0.186673,0.686469,0.226772,0.489364,0.263998,0.559009
12HundredBaud(262),0.198802,0.253485,0.219097,0.208292,0.692752,0.375341,0.615313,0.671082,0.528059,0.529853,...,0.434425,0.433500,0.172135,0.650011,0.242207,0.678973,0.285658,0.570577,0.350053,0.626016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zomba(192),0.191847,0.230431,0.188738,0.181150,0.601526,0.312474,0.553202,0.571211,0.445694,0.483333,...,0.367381,0.364765,0.147617,0.542405,0.229552,0.575534,0.279144,0.504191,0.279034,0.546504
zombywoof(537),0.238678,0.290854,0.254477,0.287354,0.808183,0.460733,0.981919,0.991112,0.627150,0.642091,...,0.423503,0.569184,0.234937,0.896709,0.317656,0.964438,0.298152,0.714429,0.386736,0.791926
"zvikar(12,403)",0.908148,1.307133,0.844710,0.907529,1.265683,1.237255,1.407068,1.379735,1.250566,1.274373,...,1.213729,1.168703,0.769592,1.356517,0.969815,1.371376,0.995705,1.363053,1.099607,1.233762


In [19]:
# 우리가 예측한 평점과 실제 평점간의 차이를 MSE로 계산
def get_mse(pred, actual):
    # 평점이 있는 실제 맥주만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE: ', 
      get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE:  3.4611927330873584


TOP N에 기반한 추천

In [20]:
# 3개의 col까지만. 3개의 맥주에 대해서 유사도가 큰 5개 선택
top_n_items = [np.argsort(item_sim_df.values[:,3])[:-5:-1]]
top_n_items


# 따라서 가장 비슷한 유사도를 가지는 맥주만 유사도 벡터로 사용
# 특정 맥주와 비슷한 유사도를 가지는 맥주 Top_N에 대해서만 적용 -> 시간오래걸림

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 맥주 개수만큼 루프
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개의 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점 계산 : 각 col 맥주별(1개), 2665명의 사용자들의 예측평점
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col,:][top_n_items].dot(
            ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(item_sim_arr[col,:][top_n_items])

    return pred

ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=10)
print('아이템 기반 최근접 TOP-N 이웃 MSE: ', 
      get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns=ratings_matrix.columns)

ratings_pred_matrix

아이템 기반 최근접 TOP-N 이웃 MSE:  1.2581976119843084


맥주,ARK Be High IPA,ARK Black Swan,ARK Hug Me,Amazing Shocking Stout,Asahi Super Dry,Asahi Super Dry Black,Ballast Point Sculpin IPA,Beck's,Berliner Kindl Jubiläums Pilsener Premium,Bintang Pilsener,...,Stephans Bräu Premium Pilsener,Suntory The Premium Malt's,The Hand and Malt Slow IPA,Tiger Beer,Tiger Radler Lemon,Tsingtao,Tsingtao Wheat Beer,Victoria Bitter,Volfas Engelman Grünberger Hefeweizen,Warsteiner Premium Verum
아이디,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.000000,0.000000,0.000000,0.00000,0.571746,0.000000,0.286343,2.283593,0.215237,0.168457,...,0.087655,0.166721,0.000000,1.639074,0.000000,1.913351,0.000000,0.799843,0.000000,0.663836
"00cobraR(1,103)",0.000000,0.000000,0.000000,0.00000,0.515088,0.000000,1.210706,1.971447,0.190402,0.534842,...,0.039445,0.140045,0.000000,1.706969,0.000000,1.976798,0.000000,0.690410,0.000000,1.001893
0o9i(439),0.000000,0.000000,0.000000,0.00000,0.208714,0.000000,1.562980,1.745868,0.303977,0.210572,...,0.061358,0.000000,0.000000,1.008103,0.000000,1.124362,0.000000,0.214527,0.000000,0.803823
1010(309),0.000000,0.000000,0.000000,0.00000,0.305091,0.000000,1.179761,1.688987,0.165567,0.262381,...,0.000000,0.000000,0.000000,1.040584,0.000000,1.284559,0.000000,0.656904,0.000000,0.968688
12HundredBaud(262),0.000000,0.000000,0.000000,0.00000,0.993122,0.149026,0.597123,1.588337,0.602439,0.466119,...,0.121528,0.196578,0.000000,1.238231,0.000000,1.273028,0.156711,0.547913,0.184478,1.042716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zomba(192),0.000000,0.000000,0.000000,0.00000,0.840391,0.142546,0.700381,0.863289,0.287312,0.819864,...,0.230294,0.000000,0.000000,0.809599,0.000000,0.843326,0.089549,0.988124,0.000000,1.000811
zombywoof(537),0.000000,0.000000,0.000000,0.00000,0.536797,0.000000,1.206539,2.199122,0.339072,0.168457,...,0.043827,0.190435,0.000000,1.859038,0.097973,2.060080,0.000000,0.967823,0.000000,0.983721
"zvikar(12,403)",0.488014,1.099362,0.564236,0.41721,1.194681,1.340147,1.858855,1.788982,1.159779,1.719909,...,1.134744,1.246943,0.408145,1.368511,0.834813,1.202172,0.644862,2.069225,0.578728,1.202959


실제 사용자에게 추천하기

In [22]:
username='007lund(92)'

# 특정 유저 대상으로 맥주추천
user_rating_id = ratings_matrix.loc[username, :]
# 유저가 먹었던 맥주들 출력
user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

맥주
Guinness Draught              3.7
Paulaner Hefe-Weissbier       3.3
Pilsner Urquell               3.1
Blue Moon Belgian White       3.0
Stella Artois                 2.9
Beck's                        2.6
Tsingtao                      2.5
Budweiser                     2.0
Heineken                      2.0
Miller Genuine Draft (MGD)    2.0
Name: 007lund(92), dtype: float64

In [23]:
# 사용자가 안 먹어본 맥주를 추천하자.
def get_not_tried_beer(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 맥주 정보를 추출해 Series로 반환
    # 반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[userId, :]

    # user_rating이 0보다 크면 기존에 관란함 영화.
    # 대상 인덱스를 추출해 list 객체로 만듦
    tried = user_rating[user_rating>0].index.tolist()

    # 모든 맥주명을 list 객체로 만듦
    beer_list = ratings_matrix.columns.tolist()

    # list comprehension으로 tried에 해당하는 영화는 beer_list에서 제외
    not_tried = [beer for beer in beer_list if beer not in tried]

    return not_tried

# 예측 평점 DataFrame에서 사용자 id 인덱스와 not_tried로 들어온 맥주명 추출 후
# 가장 예측 평점이 높은 순으로 정렬
def recomm_beer_by_userid(pred_df, userId, not_tried, top_n):
    recomm_beer = pred_df.loc[userId, not_tried].sort_values(ascending=False)[:top_n]
    return recomm_beer

# 유저가 먹지 않은 맥주이름 추출
not_tried = get_not_tried_beer(ratings_matrix, username)
not_tried

['ARK Be High IPA',
 'ARK Black Swan',
 'ARK Hug Me',
 'Amazing Shocking Stout',
 'Asahi Super Dry',
 'Asahi Super Dry Black',
 'Ballast Point Sculpin IPA',
 'Berliner Kindl Jubiläums Pilsener Premium',
 'Bintang Pilsener',
 'Blue Girl (5.0%)',
 'Cass Fresh',
 'Cass Lemon',
 'Cass Light',
 'Castlemaine XXXX Gold Lager',
 'Desperados',
 'Desperados Lime',
 'Desperados Mojito',
 'Desperados Red',
 'Edelweiss Weissbier Snowfresh',
 'Egger Märzenbier',
 'Erdinger Weissbier',
 'Erdinger Weissbier Dunkel',
 'Franziskaner Hefe-Weissbier / Weissbier Naturtrub',
 'Gambrinus Originál 10° (Desítka)',
 'Goose Island 312 Urban Wheat Ale',
 'Goose Island Goose IPA',
 'Grimbergen Cuvée Blanche',
 'Grolsch Premium Lager',
 'Guinness Original 4.2% (Ireland/UK)',
 'Harbin Beer',
 'Heineken Dark Lager',
 'Heretic Evil Twin',
 'Hite Extra Cold',
 'Hite Prime Max',
 'Hoegaarden',
 'Hoegaarden Rosée',
 'Hollandia',
 'Holsten Premium Bier',
 'Hop House 13 Lager',
 'Jeju Wit Ale',
 'Kirin Ichiban',
 'Kloud Fi

In [24]:
# 최종적으로 아이템 기반의 최근접 이웃 CF로 맥주 추천을 진행합니다.

# top_n과 비슷한 맥주만 추천에 사용
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=5)

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)

# 유저가 먹지 않은 맥주이름 추출
not_tried = get_not_tried_beer(ratings_matrix, username)

# 아이템 기반의 최근접 이웃 CF로 맥주 추천
recomm_beer = recomm_beer_by_userid(ratings_pred_matrix, username, not_tried, top_n=3)
recomm_beer = pd.DataFrame(data=recomm_beer.values, index=recomm_beer.index,
                           columns=['예측평점'])
recomm_beer

,예측평점
맥주,
Hoegaarden,2.452493
Kirin Ichiban,1.468797
Peroni Nastro Azzurro,1.246881
